# ArcGIS Online Web Map Service Validation Audit

Description: This script will check the urls of map services within webmaps to determine if they are valid. 
If they are not valid, it will write the results to a csv file so they can be taken care of. 
 
Created on: 9/3/2018
 
Purpose: This script will go through an entire ArcGIS Online Organization or a Portal Organization and look through all of the Web Maps. Then, this script will check the all of the urls of all of the map services within each Web Map to determine if they are valid. If they are not valid, it will write the results to a csv file so they can be taken care of. The csv file can then be used to aid the administrator in the cleanup of the map services with invalid urls.  This is a Jupyter Notebook written using the ArcGIS Python API. 
 
Authored by: Joe Guzi
 
Previous Production Date: 12/26/2018      

Production Date: 8/7/2024

Note:
 - 8/7/2024 - Updated with my template

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Use one of the methods below to conenct to ArcGIS Online or ArcGIS Enterprise.

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password? ")

# Connection 
gis = GIS(Organization, User, Password)
gis
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Functions (Optional)

In [ ]:
print("Begin Function Section.")
# This is where you define Functions

print("Function Section Complete!")

# Step 4: Execute Code Section

In [ ]:
print("Begin Execute Code Section.") 
Audit = gis.content.search(query="", item_type="Web Map", max_items = 10000)
s = ""
for Item in Audit:
    WebMapItem = WebMap(webmapitem = Item)
    Layers = WebMapItem.layers
    for Layer in Layers:
        URL = Layer.get('url')
        if URL != None:
            try:
                request = requests.get(URL)
                if request.status_code == 200:
                    print('Web site exists')
                else:
                    ItemID = str(Item.get("id"))
                    ItemName = str(Item.get("name"))
                    ItemTitle = str(Item.get("title"))#.replace(',', ""))
                    ItemType = str(Item.get("type"))
                    ItemOwner = str(Item.get("owner"))
                    Layername = Layer.get("title")
                    URLStatusCode = str(request.status_code)
                    URL = Layer.get('url')
                    s += ItemID + "," # switch the t to s
                    ItemTitle = ItemTitle.replace(",", ";")
                    s += ItemTitle.replace('℠', "") + ","
                    s += ItemType + ","
                    s += ItemOwner + ","
                    s += Layername + ","
                    s += URLStatusCode + ","
                    s += URL + ","
                    s += "\n"
            except:
                print('Web site does not exists')
                ItemID = str(Item.get("id"))
                ItemName = str(Item.get("name"))
                ItemTitle = str(Item.get("title"))#.replace(',', ""))
                ItemType = str(Item.get("type"))
                ItemOwner = str(Item.get("owner"))
                Layername = Layer.get("title")
                URL = Layer.get('url')
                s += ItemID + "," # switch the t to s
                ItemTitle = ItemTitle.replace(",", ";")
                s += ItemTitle.replace('℠', "") + ","
                s += ItemType + ","
                s += ItemOwner + ","
                s += Layername + ","
                s += "Error" + ","
                s += URL + ","
                s += "\n"
                
print("Execute Code Section Complete!") 

# Step 5: Write the Results to Excel (Optional)

In [ ]:
print("Begin Write Results to Excel Section.")
filename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
csvfile = filename + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Wrtie data To CSV File")vlidatelayers
    output.write("WebMapId,WebMapTitle,Type,WebMapOwner,LayerName,LayerStatus,LayerUrl\n")
    output.writelines(s)
    print("Wrtie data To CSV File Complete!")